In [33]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew
from sklearn.decomposition import PCA
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from matplotlib import pyplot as plt
from sklearn.preprocessing import RobustScaler

In [34]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [35]:
train = pd.read_csv('/content/gdrive/MyDrive/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/test.csv')
testOriginal = pd.read_csv('/content/gdrive/MyDrive/test.csv')

In [36]:
train.shape

(181507, 272)

In [37]:
test.drop('row ID', axis=1, inplace=True)

<h3>SubArea Removal<h3>

In [38]:
test.drop('sub_area', axis=1, inplace=True)
train.drop('sub_area', axis=1, inplace=True)

<h1>Categorical To Numerical<h1>

<h3>OneHot<h3>

In [39]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

<h1>Working<h1>

In [40]:
X = train.loc[:, train.columns != 'price_doc']
y = train[['price_doc']]

# scaler = RobustScaler()
# X = scaler.fit_transform(X)
# test = scaler.fit_transform(test)

In [41]:
X.shape

(181507, 287)

In [42]:
test.shape

(77789, 287)

<h1>P-Value Selection<h1>

In [43]:
# # Convert scaled_df1 to a DataFrame without specifying columns
# scaled_df1_df = pd.DataFrame(scaled_df1)

X_p = X.astype(float)

# Add a constant term to the feature matrix
X_with_const = sm.add_constant(X_p)

# Fit a linear regression model
model = sm.OLS(y, X_with_const).fit()

# Get p-values for each feature
p_values = model.pvalues[1:]  # Exclude the constant term

# Set your desired threshold for p-value
threshold = 0.00001

# Filter features based on p-value
selected_features = p_values[p_values < threshold].index

# Display selected features
print("Selected Features:")
print(selected_features)
print(len(selected_features))

# Select columns in the DataFrame
X2 = X[selected_features]
test2 = test[selected_features]


pca = PCA(n_components=10)
principalComponents = pca.fit_transform(X2)
X2 = pd.DataFrame(data = principalComponents)

pca2 = PCA(n_components=10)
principalComponents = pca2.fit_transform(test2)
test2 = pd.DataFrame(data = principalComponents)

# X = X.astype(float)
# X = sm.add_constant(X)
# mod = sm.OLS(y, X)
# res = mod.fit()
# pvalues = res.pvalues
# columns = pvalues[pvalues<0.05].index
# print(columns)
# print(len(columns))

# # Select columns in the DataFrame
# X2 = X[columns]
# test2 = test[columns]

Selected Features:
Index(['full_sq', 'life_sq', 'floor', 'children_preschool',
       'preschool_education_centers_raion',
       'school_education_centers_top_20_raion', 'healthcare_centers_raion',
       'university_top_20_raion', 'male_f', '0_6_female',
       'build_count_monolith', 'raion_build_count_with_builddate_info',
       'build_count_1971-1995', 'build_count_after_1995', 'kindergarten_km',
       'green_zone_km', 'industrial_km', 'water_treatment_km', 'water_km',
       'mkad_km', 'sadovoe_km', 'big_road2_km', 'nuclear_reactor_km',
       'swim_pool_km', 'basketball_km', 'church_synagogue_km', 'catering_km',
       'green_part_500', 'prom_part_500', 'trc_sqm_500',
       'cafe_count_500_price_1000', 'cafe_count_500_price_1500',
       'cafe_count_500_price_4000', 'cafe_count_500_price_high',
       'mosque_count_500', 'leisure_count_500', 'market_count_500',
       'green_part_1000', 'prom_part_1000', 'office_count_1000',
       'office_sqm_1000', 'trc_count_1000', 'trc_sq

<h1> Poly Interaction ONNNNNNNNN<h1>

In [44]:
poly = PolynomialFeatures(2)
X3 = poly.fit_transform(X2)
test3 = poly.fit_transform(test2)




<h1>Applying Model<h1>

In [45]:
X3.shape

(181507, 66)

In [46]:
test3.shape

(77789, 66)

In [47]:
# Create and train the Lasso regression model
alpha_value = 0.5  # You can adjust the alpha parameter based on your needs
lasso_reg = Lasso(alpha=alpha_value, random_state=42)
lasso_reg.fit(X3, y)

# Print the coefficients and intercept
print("Coefficients:", lasso_reg.coef_)
print("Intercept:", lasso_reg.intercept_)

# Make predictions on the test data
y_pred_lasso = lasso_reg.predict(test3)

Coefficients: [ 0.00000000e+00  3.48263533e+01  1.90425994e+01  2.13353071e+01
  1.63573071e+01  1.92957995e+01  5.41601031e+02  9.57703571e+02
  6.50125550e+03  1.52467506e+04  3.03772106e+02 -7.69193813e-06
 -8.81748735e-06 -9.32564310e-06 -7.10909798e-06 -5.74708869e-06
 -1.43997554e-04 -3.85617262e-04 -2.99005735e-03 -7.60876915e-03
  1.90884269e-04 -2.07954286e-06 -4.69857847e-06 -5.72047524e-06
 -8.85815032e-06  5.35131174e-06 -1.10500853e-04 -1.92901463e-03
 -3.58226872e-03 -8.12492070e-04 -2.78523939e-06 -5.53857014e-06
 -7.95433727e-06 -1.70025894e-04 -1.17601139e-04 -2.14010438e-03
 -5.26429842e-03  1.08547009e-03 -6.79071219e-07 -2.05688189e-05
 -1.80741662e-04 -2.97808314e-04 -1.63649460e-03 -3.39272905e-03
 -2.78266886e-04 -2.50372814e-05 -4.04941892e-04 -4.62797326e-04
 -1.68666208e-03 -4.27513621e-03  1.08593180e-03 -1.51437483e-02
 -7.46151023e-02 -5.24605047e-02 -1.78749493e-01  3.58954226e-02
  3.66546869e-02 -8.44695009e-02 -2.40887662e-01 -4.98718914e-02
 -3.8969697

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.653e+19, tolerance: 8.652e+15
  model = cd_fast.enet_coordinate_descent(


In [48]:
# Save the predictions to a CSV file
result_df_lasso = pd.DataFrame({'row ID': testOriginal['row ID'], 'price_doc': y_pred_lasso.flatten()})
result_df_lasso.to_csv('/content/gdrive/MyDrive/Day_6_Abdullah_Maqsood_24448_Submission_5.csv', index=False)
